In [1]:
from os import walk
from tqdm import tqdm
from PIL import Image
import torch
from tqdm import tqdm
import glob
import numpy as np

In [2]:
path = "images/"

files = glob.glob(path + "*.jpg")

In [3]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True, device='cuda:0', force_reload=True)

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to /home/kit/stud/utpqw/.cache/torch/hub/master.zip
YOLOv5 🚀 2022-11-12 Python-3.8.6 torch-1.7.1 CUDA:0 (Tesla V100-SXM2-32GB, 32511MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [4]:
images = []
for file in tqdm(files):
    image = Image.open(file)
    images.append(image)

100%|██████████| 33344/33344 [00:08<00:00, 3943.24it/s]


In [5]:
batch_size = 200
batches = np.arange(0, len(images), batch_size)

In [7]:
results = []
for x in tqdm(range(len(batches) - 1)):
    results += model(images[batches[x]:batches[x+1]], size=80).pandas().xyxy

100%|██████████| 166/166 [03:04<00:00,  1.11s/it]


In [16]:
labels = {}
for x in range(len(files)):
    name = files[x].split('/')[-1]
    labels[name] = {}
    labels[name]['labels'] = []
    for row in results[x].iterrows():
        if row[-1]['confidence'] > 0.65:
            labels[name]['labels'].append(row[-1]['name'])

IndexError: list index out of range

In [8]:
categories = set()

for image in labels.keys():
    for label in labels[image]['labels']:
        categories.add(label)

In [9]:
categories

{'airplane',
 'bed',
 'boat',
 'bottle',
 'car',
 'clock',
 'kite',
 'laptop',
 'person',
 'refrigerator',
 'sink',
 'toilet',
 'train',
 'tv'}

In [11]:
import json

with open('labels.json', 'w') as fp:
    json.dump(labels, fp)

In [12]:
categories = list(categories)
tasks = []

for category in categories: 
    task = {}
    task['label'] = category
    task['images-positive'] = []    
    task['images-negative'] = []
    
    for image in labels.keys():
        if category in labels[image]['labels']:
            task['images-positive'].append(image)
        else:
            if len(task['images-negative']) < 30:
                task['images-negative'].append(image)
                
    tasks.append(task)

In [13]:
with open('tasks.json', 'w') as fp:
    json.dump(tasks, fp)
tasks

[{'label': 'toilet',
  'images-positive': ['31016.jpg',
   '26519.jpg',
   '17658.jpg',
   '17511.jpg',
   '17474.jpg'],
  'images-negative': ['14337.jpg',
   '15529.jpg',
   '18266.jpg',
   '07425.jpg',
   '26957.jpg',
   '05648.jpg',
   '32252.jpg',
   '23088.jpg',
   '12131.jpg',
   '13794.jpg',
   '03344.jpg',
   '06848.jpg',
   '30328.jpg',
   '15810.jpg',
   '28164.jpg',
   '20304.jpg',
   '29008.jpg',
   '07788.jpg',
   '25865.jpg',
   '30623.jpg',
   '09853.jpg',
   '08668.jpg',
   '33514.jpg',
   '21007.jpg',
   '32328.jpg',
   '05021.jpg',
   '04617.jpg',
   '21296.jpg',
   '03469.jpg',
   '05284.jpg']},
 {'label': 'car',
  'images-positive': ['12243.jpg', '07583.jpg'],
  'images-negative': ['14337.jpg',
   '15529.jpg',
   '18266.jpg',
   '07425.jpg',
   '26957.jpg',
   '05648.jpg',
   '32252.jpg',
   '23088.jpg',
   '12131.jpg',
   '13794.jpg',
   '03344.jpg',
   '06848.jpg',
   '30328.jpg',
   '15810.jpg',
   '28164.jpg',
   '20304.jpg',
   '29008.jpg',
   '07788.jpg',
   '

In [14]:
import numpy as np

In [15]:
np.arange(0, 100, 5)

array([ 0,  5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95])